# 差動駆動車のMPCでの軌道追従制御プログラムの実装

## 差動駆動車のダイナミクス
差動駆動車のダイナミクスは、
$$\begin{aligned}
\dot{x} &= v \cos \theta \\
\dot{y} &= v \sin \theta \\
\dot{\theta} &= \omega
\end{aligned}$$
で、ここで、
$$\begin{aligned}
v &= \frac{v_L + v_R}{2} & \\
\omega &= \frac{v_R - v_L}{2l} \cos \alpha & (\alpha:\text{車輪の相対位置を決める角度},\; l\text{:車輪間の距離})
\end{aligned}
$$
である。ここで、$v_L,v_R$が独立変数で、制御量である。

In [2]:
import numpy as np
from typing import List, Tuple

class DiffCar:
	"""差動駆動車のクラス"""
	def __init__(self,wheel_distance,wheel_angle,x_init=0,y_init=0,theta_init=0):
		self.wheel_distance = wheel_distance
		self.wheel_angle = wheel_angle
		self.x = x_init
		self.y = y_init
		self.theta = theta_init
  
	def x_dot(self,x,y,theta,v_R,v_L):
		v = (v_R + v_L)/2
		x_dot = v*np.cos(theta)
		return x_dot

	def y_dot(self,x,y,theta,v_R,v_L):
		v = (v_R + v_L)/2
		y_dot = v*np.sin(theta)
		return y_dot

	def theta_dot(self,x,y,theta,v_R,v_L):
		v = (v_R + v_L)/2
		omega = (v_R - v_L)/self.wheel_distance *np.cos(self.wheel_angle)
		theta_dot = omega
		return theta_dot

	def dynamics(self,v_R,v_L) -> Tuple[float, float, float]:
		"""
		車輪の速度をv_R,v_Lとしたときの運動方程式を返す

		Parameters
		----------
		v_R : float
			 右車輪の速度
		v_L : float
			 左車輪の速度

		Returns
		-------
		Tuple[float, float, float]
			車の速度と角速度 
		"""
  
		v = (v_R + v_L)/2
		omega = (v_R - v_L)/self.wheel_distance *np.cos(self.wheel_angle)
		x_dot = v*np.cos(self.theta)
		y_dot = v*np.sin(self.theta)
		theta_dot = omega
		return x_dot,y_dot,theta_dot

	def runge_kutta_step(self,x, y, theta, dt, v_R, v_L):
		k1_x = dt * self.x_dot(x, y, theta, v_R, v_L)
		k1_y = dt * self.y_dot(x, y, theta, v_R, v_L)
		k1_theta = dt * self.theta_dot(x, y, theta, v_R, v_L)

		k2_x = dt * self.x_dot(x + 0.5 * k1_x, y + 0.5 * k1_y, theta + 0.5 * k1_theta, v_R, v_L)
		k2_y = dt * self.y_dot(x + 0.5 * k1_x, y + 0.5 * k1_y, theta + 0.5 * k1_theta, v_R, v_L)
		k2_theta = dt * self.theta_dot(x + 0.5 * k1_x, y + 0.5 * k1_y, theta + 0.5 * k1_theta, v_R, v_L)

		k3_x = dt * self.x_dot(x + 0.5 * k2_x, y + 0.5 * k2_y, theta + 0.5 * k2_theta, v_R, v_L)
		k3_y = dt * self.y_dot(x + 0.5 * k2_x, y + 0.5 * k2_y, theta + 0.5 * k2_theta, v_R, v_L)
		k3_theta = dt * self.theta_dot(x + 0.5 * k2_x, y + 0.5 * k2_y, theta + 0.5 * k2_theta, v_R, v_L)

		k4_x = dt * self.x_dot(x + k3_x, y + k3_y, theta + k3_theta, v_R, v_L)
		k4_y = dt * self.y_dot(x + k3_x, y + k3_y, theta + k3_theta, v_R, v_L)
		k4_theta = dt * self.theta_dot(x + k3_x, y + k3_y, theta + k3_theta, v_R, v_L)

		x_next = x + (k1_x + 2*k2_x + 2*k3_x + k4_x) / 6.0
		y_next = y + (k1_y + 2*k2_y + 2*k3_y + k4_y) / 6.0
		theta_next = theta + (k1_theta + 2*k2_theta + 2*k3_theta + k4_theta) / 6.0

		return x_next, y_next, theta_next

	def step(self,v_R,v_L,dt):
		"""
		車輪の速度をv_R,v_Lとして、dt時間だけ車を動かして、位置と姿勢を更新する

		Parameters
		----------
		v_R : float
				右車輪の速度
		v_L : float
				左車輪の速度
		dt : float
				時間
		"""
		#４次ルンゲクッタ法を用いて、位置と姿勢を更新
		self.x, self.y,self.theta = self.runge_kutta_step(self.x, self.y, self.theta, dt, v_R, v_L)

		return self.x, self.y, self.theta


In [18]:
# 実際に動かしてその軌道をgifにする
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import FancyArrow
from PIL import Image
import io

def create_gif(x_list, y_list, theta_list, dt, fps):
    num_frames = int(len(x_list) // (fps * dt))

    pil_images = []
    x_list_range = max(x_list) - min(x_list)
    y_list_range = max(y_list) - min(y_list)
    xlim_min = min(x_list) - 0.5 * x_list_range
    xlim_max = max(x_list) + 0.5 * x_list_range
    ylim_min = min(y_list) - 0.5 * y_list_range
    ylim_max = max(y_list) + 0.5 * y_list_range

    for i in range(num_frames):

        fig, ax = plt.subplots()
        # グラフの縦横比を揃えます
        ax.set_aspect('equal')
        # 赤線で軌跡を描画します
        idx = int(i * (fps * dt))
        ax.plot(x_list[:idx], y_list[:idx], color='red')

        # 車の向きを矢印で描画します
        dx = np.cos(theta_list[idx])*min(y_list_range,x_list_range) *0.1
        dy = np.sin(theta_list[idx])*min(y_list_range,x_list_range) *0.1
        arrow = FancyArrow(x_list[idx], y_list[idx], dx, dy, width=min(y_list_range,x_list_range)*0.02, color='blue')
        ax.add_patch(arrow)

        # XとYの軸を調整します
        ax.set_xlim([xlim_min, xlim_max])
        ax.set_ylim([ylim_min, ylim_max])

        # イメージをバイト型で保存します
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)

        # PIL Imageとしてロードしてリストに追加します
        pil_image = Image.open(buf)
        pil_images.append(pil_image)
        plt.close(fig)

    # PIL Imageとして保存します
    pil_images[0].save('output.gif',
                   save_all=True, append_images=pil_images[1:], optimize=False, duration=dt*1000, loop=0)

#車のパラメータ
l = 0.02 #車輪間の距離[m]
alpha = np.pi/6 #車輪の角度[rad]

car = DiffCar(l,alpha,0.1,0.1,0)
dt = 0.1
total_steps =100
v_R = 0.03
v_L = 0.01

x_list = []
y_list = []
theta_list = []
for i in range(total_steps):
   x,y,theta = car.step(v_R,v_L,dt)
   x_list.append(x)
   y_list.append(y)
   theta_list.append(theta)
   
create_gif(x_list, y_list, theta_list, dt, fps=10)



## MPC用の線形ダイナミクスの導出
上式を$dt$間隔で離散化すると、
$$\begin{aligned}
x_{t+1} &= x_t + v_t \cos \theta_t dt \\
y_{t+1} &= y_t + v_t \sin \theta_t dt \\
\theta_{t+1} &= \theta_t + \omega_t dt
\end{aligned}$$
を得る。
いま、軌道追従制御を考え、理想となる軌道が与えられている状況を考える。
理想となる軌道を($x^*_{t},y^*_{t},\theta^{*}_{t}$)として、上式を$x^*_{t},y^*_{t},\theta^{*}_{t}$の周りでテイラー展開して線形化すると、

$$\begin{aligned}
x_{t+1} &= x_{t} + v_t \cos \theta^*_{t} dt - v_t \sin \theta^*_{t} (\theta_t-\theta^*_{t}) dt + \mathcal{O}(dt^2) \\
y_{t+1} &= y_{t} + v_t \sin \theta^*_{t} dt + v_t \cos \theta^*_{t} (\theta_t-\theta^*_{t}) dt + \mathcal{O}(dt^2) \\
\theta_{t+1} &= \theta_{t} + \omega_t dt
\end{aligned}$$
となる。さらに、理想となる軌道から、理想となる速度$v^*_{t}$を近似的に求めることができるため、これを用いて制御量を$v_t = v^*_{t} + u_t$とし、制御量が微小であると仮定する。すると、上式は近似的に
$$\begin{aligned}
x_{t+1} &= x_{t} + (v^*_{t} + u_t) \cos \theta^*_{t} dt - v^*_{t} \sin \theta^*_{t} (\theta_t-\theta^*_{t}) dt + \mathcal{O}(dt^2) \\
y_{t+1} &= y_{t} + (v^*_{t} + u_t) \sin \theta^*_{t} dt + v^*_{t} \cos \theta^*_{t} (\theta_t-\theta^*_{t}) dt + \mathcal{O}(dt^2) \\
\theta_{t+1} &= \theta_{t} + \omega_t dt
\end{aligned}$$
となる。
$$\begin{aligned}
\bm{x}_t = \begin{bmatrix}
x_{t} \\
y_{t} \\
\theta_{t}
\end{bmatrix},\;
\bm{u}_t = \begin{bmatrix}
u_{t} \\
\omega_{t}
\end{bmatrix}
\end{aligned}$$
とおいて、上式を整理して行列で表すと、
$$\begin{aligned}
\bm{x}_{t+1}
&=
\begin{bmatrix}
1 & 0 & -v^*_{t} \sin \theta^*_{t} dt \\
0 & 1 & v^*_{t} \cos \theta^*_{t} dt \\
0 & 0 & 1
\end{bmatrix}
\bm{x}_{t}
+
dt
\begin{bmatrix}
\cos \theta^*_{t} & 0 \\
\sin \theta^*_{t} & 0 \\
0 & 1
\end{bmatrix}
\bm{u}_{t}
+
v_t^* dt
\begin{bmatrix}
\cos \theta^*_{t} + \sin \theta^*_{t} \theta^*_{t} \\
\sin \theta^*_{t} - \cos \theta^*_{t} \theta^*_{t} \\
0
\end{bmatrix}
\end{aligned}$$
となる。上式をMPC用の線形ダイナミクスとして用いる。